In [1]:
import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../src/'))

%load_ext autoreload
%autoreload 2

In [25]:
from attr import dataclass
from datetime import date, timedelta
from enum import Enum
import pandas as pd

In [23]:
from markets_insights.datareader import data_reader

data = data_reader.NseIndicesReader().read(data_reader.DateRangeCriteria(date(2023, 1, 1), date(2023, 1, 31)))

In [46]:
reader = data_reader.MemoryCachedDataReader(data_reader.NseIndicesReader())
data = reader.read( data_reader.DateRangeCriteria(date(2019, 1, 1), date(2019, 1, 31)) )
data = reader.read( data_reader.DateRangeCriteria(date(2020, 3, 1), date(2020, 3, 15)) )

In [47]:
reader.read_data( data_reader.DateRangeCriteria(date(2019, 1, 1), date(2020, 3, 15)) )

,Identifier,Index Date,Open,High,Low,Close,Points Change,Change(%),Volume,Turnover,P/E,P/B,Div Yield,Date,PreviousClose
0,Nifty 50,01-01-2019,10881.70,10923.60,10807.10,10910.10,47.55,.44,159404542.0,8.688260e+10,26.28,3.4,1.24,2019-01-01,NaN
1,Nifty Next 50,01-01-2019,28243.95,28252.20,27988.60,28111.65,-67.15,-.24,100916543.0,2.354600e+10,42.32,3.96,.87,2019-01-01,NaN
2,Nifty 100,01-01-2019,11130.55,11161.00,11051.50,11148.80,38.65,.35,261689768.0,1.105554e+11,27.69,3.46,1.19,2019-01-01,NaN
3,Nifty 200,01-01-2019,5766.00,5779.55,5726.70,5773.70,18.35,.32,432202773.0,1.428473e+11,28.87,3.33,1.17,2019-01-01,NaN
4,Nifty 500,01-01-2019,9186.70,9206.05,9128.40,9197.90,27.90,.3,724662248.0,1.699299e+11,29.7,3.21,1.14,2019-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,Nifty Composite G-sec Index,13-03-2020,2130.90,2130.90,2116.54,2116.54,-13.97,-.66,NaN,NaN,-,-,-,2020-03-13,2130.51
725,Nifty 10 yr Benchmark G-Sec,13-03-2020,1933.99,1933.99,1918.51,1922.59,-11.08,-.57,NaN,NaN,-,-,-,2020-03-13,1933.67
726,Nifty 10 yr Benchmark G-Sec (Clean Price),13-03-2020,965.36,965.36,957.42,959.52,-5.85,-.61,NaN,NaN,-,-,-,2020-03-13,965.37
727,Nifty 1D Rate Index,13-03-2020,1878.84,1878.84,1878.84,1878.84,0.24,.01,NaN,NaN,-,-,-,2020-03-13,1878.60


In [36]:
df = pd.DataFrame()
df['Date'] = data['Date']
df['Gap'] = data['Date'].diff().dt.days

df['NewRange'] = df['Gap'] > 3

df['Group'] = df['NewRange'].cumsum()

ranges_df = df.groupby('Group')['Date'].agg(['min', 'max']).reset_index(drop=True)

ranges: list[data_reader.DateRangeCriteria] = []
for index, row in ranges_df.iterrows():
  ranges.append(data_reader.DateRangeCriteria(row['min'].date(), row['max'].date()))
ranges


[DateRangeCriteria(from_date=datetime.date(2023, 1, 2), to_date=datetime.date(2023, 1, 31))]